In [2]:
!pip install fancyimpute
import numpy as np
import pandas as pd
from fancyimpute import IterativeImputer
from sklearn.preprocessing import StandardScaler

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.4 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29881 sha256=13ebcc052d5d0613b0b049a26c744a04a0f8a648acaeab1f3249c16d2e61fe2f
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=474726a5d8e94cf3eeec94b739d3aed11221968715aa13d79cbe5efb2fa4e6b7
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [3]:
'''
pre-processes melanoma data

'''

'\npre-processes melanoma data\n\n'

In [10]:
# read in data

raw_data = pd.read_csv('/content/total_PD_1.csv')
raw_data.tail()

<ipython-input-10-8ea765e0160b>:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('/content/total_PD_1.csv')


,Unnamed: 0,GSM2445716,GSM2445717,GSM2445718,GSM2445719,GSM2445720,GSM2445721,GSM2445722,GSM2445723,GSM2445724,...,SRR5088909,SRR5088911,SRR5088913,SRR5088914,SRR5088916,SRR5088920,SRR5088922,SRR5088924,SRR5088926,SRR5088929
56265,snoZ6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56266,snosnR66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56267,uc_338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003664563208162,0.031978130771298,0.0857459707118245,0.04071053428664,0.0512846604842121,0.158733159882816,0.18278951659712,0.0657273971922831,0.0777414803400673,0.0
56268,yR211F11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0355281824962792,0.0
56269,Response,N,N,N,N,N,N,N,N,N,...,N,R,N,N,N,N,N,R,N,R


In [19]:
# some re-formatting

# rename column

raw_data = raw_data.rename(columns={raw_data.columns[0]: 'patient'})

# transpose so columns are genes and rows are patients

t_data = raw_data.set_index('patient').T
print(t_data.head())

# split data

labels = t_data['Response']
clean_data = t_data.drop(columns = ['Response'])


patient    5S_rRNA 5_8S_rRNA  7SK A1BG A1BG-AS1 A1CF        A2M A2M-AS1 A2ML1  \
GSM2445716     NaN       NaN  NaN  NaN      NaN  NaN  12.488406     NaN   NaN   
GSM2445717     NaN       NaN  NaN  NaN      NaN  NaN  11.463665     NaN   NaN   
GSM2445718     NaN       NaN  NaN  NaN      NaN  NaN  12.110351     NaN   NaN   
GSM2445719     NaN       NaN  NaN  NaN      NaN  NaN  13.324661     NaN   NaN   
GSM2445720     NaN       NaN  NaN  NaN      NaN  NaN  14.424133     NaN   NaN   

patient    A2ML1-AS1  ... snoU2_19 snoU83B snoZ196 snoZ278 snoZ40 snoZ6  \
GSM2445716       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445717       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445718       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445719       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   
GSM2445720       NaN  ...      NaN     NaN     NaN     NaN    NaN   NaN   

patient    snosnR66 uc_338 yR211F11.2 Response  
GSM2445716   

In [24]:
# drop any genes with > 90% missing data

clean_data = clean_data.dropna(thresh = clean_data.shape[0]*0.9, axis=1)
clean_data.head()

patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,12.488406,7.015997,9.380738,6.820867,4.850411,2.640958,3.341397,11.308956,7.600959,2.962886,...,6.067222319,1.96288566,6.95384052,9.189779473,6.800828901,3.100389183,3.96288566,10.79133953,5.49893856,7.033274987
GSM2445717,11.463665,5.434654,8.657046,6.309123,4.615226,3.112725,4.228203,12.653822,9.572157,2.112725,...,4.228202656,6.794549479,6.794549479,7.434653534,6.228202656,4.434653534,3.69768794,10.21251156,7.636287395,6.142472782
GSM2445718,12.110351,5.669961,9.850125,8.105658,6.165918,2.969521,3.554484,10.517701,5.84399,2.706487,...,5.291449305,0.384558709,6.999268553,7.724408712,6.191913631,2.384558709,2.706486804,10.0284149,6.554483711,6.554483711
GSM2445719,13.324661,6.475534,8.687919,8.267292,5.913655,3.521337,3.521337,11.222933,7.775579,2.74373,...,3.521337443,2.936374942,8.332979723,8.184302455,6.379318438,6.276224945,4.258303037,10.49575203,8.498617366,5.913654865
GSM2445720,14.424133,7.225295,9.963674,8.127778,7.749266,4.215834,4.552869,11.333136,8.267115,3.775262,...,5.552869187,3.427338305,9.404618228,9.841350799,7.001329688,6.410850182,5.360224109,9.682152204,8.987497414,6.253308905


In [25]:
# log2 transformation

# note that samples from the Melanoma dataset GSE931 are already log2 transformed

# add 1 to all values for log2 transformation

clean_data = clean_data.applymap(lambda x: x+1 if isinstance(x, (int, float)) else x)

# subset GSE931 samples from rest

gse931 = clean_data[clean_data.index.str.contains('GSM24457')]
to_transform = clean_data[~clean_data.index.str.contains("GSM24457")]

# log2 transform

log2_data = to_transform.applymap(lambda x: np.log2(x) if isinstance(x, (int, float)) else x)

# re-join data

log2_data = pd.concat([gse931, log2_data], axis = 0)
log2_data.head()


patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,13.488406,8.015997,10.380738,7.820867,5.850411,3.640958,4.341397,12.308956,8.600959,3.962886,...,6.067222319,1.96288566,6.95384052,9.189779473,6.800828901,3.100389183,3.96288566,10.79133953,5.49893856,7.033274987
GSM2445717,12.463665,6.434654,9.657046,7.309123,5.615226,4.112725,5.228203,13.653822,10.572157,3.112725,...,4.228202656,6.794549479,6.794549479,7.434653534,6.228202656,4.434653534,3.69768794,10.21251156,7.636287395,6.142472782
GSM2445718,13.110351,6.669961,10.850125,9.105658,7.165918,3.969521,4.554484,11.517701,6.843990,3.706487,...,5.291449305,0.384558709,6.999268553,7.724408712,6.191913631,2.384558709,2.706486804,10.0284149,6.554483711,6.554483711
GSM2445719,14.324661,7.475534,9.687919,9.267292,6.913655,4.521337,4.521337,12.222933,8.775579,3.743730,...,3.521337443,2.936374942,8.332979723,8.184302455,6.379318438,6.276224945,4.258303037,10.49575203,8.498617366,5.913654865
GSM2445720,15.424133,8.225295,10.963674,9.127778,8.749266,5.215834,5.552869,12.333136,9.267115,4.775262,...,5.552869187,3.427338305,9.404618228,9.841350799,7.001329688,6.410850182,5.360224109,9.682152204,8.987497414,6.253308905


In [26]:
# missing data imputation

imputer = IterativeImputer()
imputed_data = imputer.fit_transform(log2_data)

imputed_data = pd.DataFrame(imputed_data, columns = log2_data.columns, index = log2_data.index)
imputed_data.head()

patient,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,AKT3,ALCAM,AMBP,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
GSM2445716,13.488406,8.015997,10.380738,7.820867,5.850411,3.640958,4.341397,12.308956,8.600959,3.962886,...,6.067222,1.962886,6.953841,9.189779,6.800829,3.100389,3.962886,10.791340,5.498939,7.033275
GSM2445717,12.463665,6.434654,9.657046,7.309123,5.615226,4.112725,5.228203,13.653822,10.572157,3.112725,...,4.228203,6.794549,6.794549,7.434654,6.228203,4.434654,3.697688,10.212512,7.636287,6.142473
GSM2445718,13.110351,6.669961,10.850125,9.105658,7.165918,3.969521,4.554484,11.517701,6.843990,3.706487,...,5.291449,0.384559,6.999269,7.724409,6.191914,2.384559,2.706487,10.028415,6.554484,6.554484
GSM2445719,14.324661,7.475534,9.687919,9.267292,6.913655,4.521337,4.521337,12.222933,8.775579,3.743730,...,3.521337,2.936375,8.332980,8.184302,6.379318,6.276225,4.258303,10.495752,8.498617,5.913655
GSM2445720,15.424133,8.225295,10.963674,9.127778,8.749266,5.215834,5.552869,12.333136,9.267115,4.775262,...,5.552869,3.427338,9.404618,9.841351,7.001330,6.410850,5.360224,9.682152,8.987497,6.253309


In [48]:
# z-score transformation (for PCA)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(imputed_data)

scaled_data = pd.DataFrame(scaled_data)
scaled_data.head()
print(scaled_data.shape)

(157, 717)


In [49]:
# export '/content/total_PD_1.csv'

# imputed log2 transformed data
imputed_data.to_csv('imputed_data.csv')

# normalized version of above^ (for PCA)
scaled_data.to_csv('scaled_data.csv')

# true labels (response y/n)
labels.to_csv('labels.csv')